# Normal OLS Regression

For the OLS Regression process below, I want to see the general effect of every variables on the change of GDP 
Especially I want to see if a drop of interest rate - federal_funds_rate can be a boost for GDP growth

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.stats as stat
import sklearn as sk3
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [2]:
import statsmodels.formula.api as sm
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
df = pd.read_csv("../Data/modelData.csv")
# Drop NaN values
df = df.dropna()
df.head(10)

,year,month,CPI,CPI_Inflation,avg_HPI,long_term_interest,federal_funds_rates,budget_on_education,gdp,population,employed_percent,unemployed_percent,lowest,second,third,fourth,top_5_percent
173,1987,1,111.2,0.63,60.603750,7.08,6.43,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
175,1987,2,111.6,0.36,61.016875,7.25,6.10,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
176,1987,3,112.1,0.45,61.311250,7.25,6.13,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
177,1987,4,112.7,0.54,61.746250,8.02,6.37,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
179,1987,5,113.1,0.35,62.362500,8.61,6.85,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
181,1987,6,113.5,0.35,62.912500,8.40,6.73,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
182,1987,7,113.8,0.26,63.451875,8.45,6.58,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
184,1987,8,114.4,0.53,63.901250,8.76,6.73,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
186,1987,9,115.0,0.52,64.386875,9.42,7.22,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517
190,1987,10,115.3,0.26,64.750000,9.52,7.29,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517


In [3]:
df.describe()

,year,month,CPI,CPI_Inflation,avg_HPI,long_term_interest,federal_funds_rates,budget_on_education,gdp,population,employed_percent,unemployed_percent,lowest,second,third,fourth,top_5_percent
count,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,2.880000e+02,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000
mean,1998.500000,6.500000,167.047392,0.238472,109.006842,5.840278,4.359132,46896.000000,9.707590e+06,209063.875000,62.479167,5.812500,23462.791667,44233.708333,69898.833333,109575.750000,192885.708333
std,6.934236,3.458061,31.172453,0.322599,40.099796,1.758579,2.441827,27294.070084,3.279140e+06,17355.912158,1.355432,1.381202,1076.134429,1767.243965,3251.975580,6875.556281,15987.531885
min,1987.000000,1.000000,111.200000,-1.920000,60.603750,2.420000,0.110000,19475.000000,4.870217e+06,182753.000000,58.500000,4.000000,21639.000000,41342.000000,64985.000000,99622.000000,167517.000000
25%,1992.750000,3.750000,142.450000,0.087500,74.126548,4.442500,2.437500,28353.750000,6.793863e+06,194329.750000,62.075000,4.850000,22687.750000,42917.750000,66763.250000,102373.750000,175536.500000
50%,1998.500000,6.500000,164.150000,0.240000,91.831905,5.720000,4.885000,33581.000000,9.374896e+06,206486.500000,62.700000,5.550000,23476.000000,44331.500000,70528.000000,112876.000000,201309.500000
75%,2004.250000,9.250000,190.925000,0.412500,136.224022,7.185000,5.800000,63600.500000,1.247963e+07,224038.250000,63.125000,6.125000,24171.250000,45663.000000,72778.750000,115515.500000,206440.250000
max,2010.000000,12.000000,219.964000,1.220000,193.566957,9.520000,9.850000,131891.000000,1.496437e+07,237830.000000,64.400000,9.600000,25580.000000,47110.000000,74475.000000,118516.000000,212081.000000


In [4]:
# The variables columns will be saved in a list
# Add a column called lgdp which is the log of all gdps and use that variable instead
# Add a column called lCPI which is the log of all CPI and use that variable instead
# Add a column called lfederal_funds_rates which is the log of all federal_funds_rate and use that variable instead

import math
lgdp = []
lCPI = []
lfederal = []

for i in df.gdp:
    lgdp.append(math.log(i))
for i in df.CPI:
    lCPI.append(math.log(i))
for i in df.federal_funds_rates:
    lfederal.append(math.log(i))
    
df["lgdp"] = lgdp
df["lCPI"] = lCPI
df["lfederal_funds_rates"] = lfederal
df.head()

use_cols = df.columns.tolist()
use_cols.remove("lgdp")
use_cols.remove("gdp")
use_cols.remove("CPI")
use_cols.remove("federal_funds_rates")
use_cols

['year',
 'month',
 'CPI_Inflation',
 'avg_HPI',
 'long_term_interest',
 'budget_on_education',
 'population',
 'employed_percent',
 'unemployed_percent',
 'lowest',
 'second',
 'third',
 'fourth',
 'top_5_percent',
 'lCPI',
 'lfederal_funds_rates']

In [5]:
df.head()

,year,month,CPI,CPI_Inflation,avg_HPI,long_term_interest,federal_funds_rates,budget_on_education,gdp,population,employed_percent,unemployed_percent,lowest,second,third,fourth,top_5_percent,lgdp,lCPI,lfederal_funds_rates
173,1987,1,111.2,0.63,60.603750,7.08,6.43,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517,15.398649,4.711330,1.860975
175,1987,2,111.6,0.36,61.016875,7.25,6.10,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517,15.398649,4.714921,1.808289
176,1987,3,112.1,0.45,61.311250,7.25,6.13,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517,15.398649,4.719391,1.813195
177,1987,4,112.7,0.54,61.746250,8.02,6.37,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517,15.398649,4.724729,1.851599
179,1987,5,113.1,0.35,62.362500,8.61,6.85,19475,4870217,182753,61.5,6.2,22356,42434,66239,100109.0,167517,15.398649,4.728272,1.924249


In [6]:
df.lgdp

173    15.398649
175    15.398649
176    15.398649
177    15.398649
179    15.398649
         ...    
563    16.521183
564    16.521183
565    16.521183
566    16.521183
567    16.521183
Name: lgdp, Length: 288, dtype: float64

In [7]:
from sklearn import preprocessing

x = df[use_cols].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_model = pd.DataFrame(x_scaled)
df_model.columns = use_cols
df_model["lgdp"] = df.lgdp.tolist()
df_model.head()

,year,month,CPI_Inflation,avg_HPI,long_term_interest,budget_on_education,population,employed_percent,unemployed_percent,lowest,second,third,fourth,top_5_percent,lCPI,lfederal_funds_rates,lgdp
0,0.0,0.000000,0.812102,0.000000,0.656338,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.000000,0.905112,15.398649
1,0.0,0.090909,0.726115,0.003107,0.680282,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.005264,0.893390,15.398649
2,0.0,0.181818,0.754777,0.005321,0.680282,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.011817,0.894482,15.398649
3,0.0,0.272727,0.783439,0.008593,0.788732,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.019643,0.903026,15.398649
4,0.0,0.363636,0.722930,0.013227,0.871831,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.024837,0.919189,15.398649


In [8]:
# Test run OLS log(gdp) on all variables
# I also include a Breusch Pagan test and a White for hetereoskedasticity for every variables. 

import math
import statsmodels.formula.api as sm

var = ""
for col in use_cols:
    var += col
    if col == 'lfederal_funds_rates':
        break
    var += " + "
total = "lgdp ~ " + var
residual = "resid ~ " + var

result = sm.ols(formula= total, data=df_model).fit()
df_model['yhat'] = result.fittedvalues
df_model['resid'] = result.resid

# Check for Breusch Pagan test for Hetereoskedasticity
name = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
result_re = sm.ols(formula=residual, data=df_model).fit()
test = sms.het_breuschpagan(result_re.resid, result_re.model.exog)

# Check for White test for Hetereoskedasticity
result_wh = sm.ols(formula="resid**2 ~ yhat + yhat**2", data=df_model).fit()
white_test = sms.het_white(result_wh.resid,  result_wh.model.exog)

print(result.params)
print(result.summary())
print("")
print("Breusch Pagan test for Hetereoskedasticity")
print(lzip(name, test))
print("White test for Hetereoskedasticity")
print(lzip(name, white_test))

Intercept               15.304505
year                     0.907602
month                   -0.012309
CPI_Inflation           -0.002232
avg_HPI                  0.119875
long_term_interest      -0.004985
budget_on_education     -0.030810
population              -0.017703
employed_percent         0.205433
unemployed_percent       0.083192
lowest                   0.025201
second                  -0.002168
third                   -0.065461
fourth                   0.062893
top_5_percent           -0.004957
lCPI                     0.187748
lfederal_funds_rates    -0.036178
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                   lgdp   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.148e+05
Date:                Tue, 19 May 2020   Prob (F-statistic):               0.00
Time:       

# Divide Data and check for error

In [9]:
# As seen above, even tho there is no heteroskedasticity error, 
# but I think there can be a collinearity error since R squared is a perfect 1.
# Thus, I would try to separate the dataset to 2 parts, 1 with variables that have very high R squared (> 0.7) 
# and one with lower R squared (> 0.1 and < 0.7) to avoid cullinearity error between variables

In [10]:
# I created a loop to check for OLS R squared values when I regress log(gdp) on single variables
# I want to try separate variables that have of too high single R squared and those with lowered R squared to see if there is a collinearity error for the higher ones 
# I also include a Breusch Pagan test for hetereoskedasticity for every variables. 
# I would have to ommit variables that has hetereoskedasticity signs

significant = {}
new_use = {}
hetereo = {};

for col in use_cols:
    command = "lgdp ~ " + col
    residual = "resid ~ " + col
    result = sm.ols(formula=command, data=df_model).fit()
    df_model['yhat'] = result.fittedvalues
    df_model['resid'] = result.resid
    
    if result.rsquared > 0.7:
        significant[col] = result.rsquared
    elif 0.1 < result.rsquared < 0.7:
        new_use[col] = result.rsquared
    
    # Check for Breusch Pagan test for Hetereoskedasticity
    result_re = sm.ols(formula=residual, data=df_model).fit()
    test = sms.het_breuschpagan(result_re.resid, result_re.model.exog)
    if test[0] < 0.05:
        hetereo[col] = test[0]
    print(col)
    print('R2: ', result.rsquared)
    print('F: ', result.fvalue)
    print("Breusch Pagan test for Hetereoskedasticity")
    print(lzip(name, test))
    print("")

year
R2:  0.9918785433352868
F:  34929.35751617514
Breusch Pagan test for Hetereoskedasticity
[('Lagrange multiplier statistic', 62.46775692440013), ('p-value', 2.708427438189514e-15), ('f-value', 79.21607233068536), ('f p-value', 6.541400878612166e-17)]

month
R2:  0.0
F:  0.0
Breusch Pagan test for Hetereoskedasticity
[('Lagrange multiplier statistic', 0.0), ('p-value', 1.0), ('f-value', 0.0), ('f p-value', 1.0)]

CPI_Inflation
R2:  0.029442793617708962
F:  8.676087220095262
Breusch Pagan test for Hetereoskedasticity
[('Lagrange multiplier statistic', 32.67254400254401), ('p-value', 1.0906702174067936e-08), ('f-value', 36.59750397082517), ('f p-value', 4.536506184317832e-09)]

avg_HPI
R2:  0.7948578678487346
F:  1108.1553448860157
Breusch Pagan test for Hetereoskedasticity
[('Lagrange multiplier statistic', 6.073039506395492), ('p-value', 0.013726055901573381), ('f-value', 6.160777585045864), ('f p-value', 0.01363550355866779)]

long_term_interest
R2:  0.8523315602062751
F:  1650.771

In [11]:
# Variables with high singular R squared
significant

{'year': 0.9918785433352868,
 'avg_HPI': 0.7948578678487346,
 'long_term_interest': 0.8523315602062751,
 'population': 0.9817873668708964,
 'fourth': 0.8079598978941911,
 'top_5_percent': 0.8559958224659169,
 'lCPI': 0.9879450206812289}

In [12]:
# Variables with lower singular R squared
new_use

{'budget_on_education': 0.6581226017062498,
 'lowest': 0.32311251566361043,
 'second': 0.3601038539045589,
 'third': 0.5822818630555828,
 'lfederal_funds_rates': 0.38765522636746064}

In [13]:
# Variables with signs of hetereoskedasticity
hetereo

{'month': 0.0}

In [14]:
df_model.head()

,year,month,CPI_Inflation,avg_HPI,long_term_interest,budget_on_education,population,employed_percent,unemployed_percent,lowest,second,third,fourth,top_5_percent,lCPI,lfederal_funds_rates,lgdp,yhat,resid
0,0.0,0.000000,0.812102,0.000000,0.656338,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.000000,0.905112,15.398649,15.880847,-0.482198
1,0.0,0.090909,0.726115,0.003107,0.680282,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.005264,0.893390,15.398649,15.891598,-0.492949
2,0.0,0.181818,0.754777,0.005321,0.680282,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.011817,0.894482,15.398649,15.890597,-0.491948
3,0.0,0.272727,0.783439,0.008593,0.788732,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.019643,0.903026,15.398649,15.882760,-0.484111
4,0.0,0.363636,0.722930,0.013227,0.871831,0.0,0.0,0.508475,0.392857,0.181934,0.18932,0.132139,0.025775,0.0,0.024837,0.919189,15.398649,15.867937,-0.469288


# Rerun OLS regression on 2 separated datasets 

In [15]:
# OLS Regression on the variables that have lower R-squared values 

var = ""
for col in new_use.keys():
    var += col
    li = list(new_use.keys())
    if col == li[-1]:
        break
    var += " + "
total = "lgdp ~ " + var
residual = "resid ~ " + var

result = sm.ols(formula= total, data=df_model).fit()
df_model['yhat'] = result.fittedvalues
df_model['resid'] = result.resid

# Check for Breusch Pagan test for Hetereoskedasticity
result_re = sm.ols(formula=residual, data=df_model).fit()
test = sms.het_breuschpagan(result_re.resid, result_re.model.exog)

# Check for White test for Hetereoskedasticity
result_wh = sm.ols(formula="resid**2 ~ yhat + yhat**2", data=df_model).fit()
white_test = sms.het_white(result_wh.resid,  result_wh.model.exog)


print(result.params)
print(result.summary())
print("")
print("Breusch Pagan test for Hetereoskedasticity")
print(lzip(name, test))
print("White test for Hetereoskedasticity")
print(lzip(name, white_test))

Intercept               15.768080
budget_on_education      0.471683
lowest                  -0.485380
second                  -0.800799
third                    1.657687
lfederal_funds_rates    -0.116729
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                   lgdp   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     586.4
Date:                Tue, 19 May 2020   Prob (F-statistic):          1.09e-146
Time:                        02:32:25   Log-Likelihood:                 242.92
No. Observations:                 288   AIC:                            -473.8
Df Residuals:                     282   BIC:                            -451.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                 

In [16]:
# OLS Regression on the variables that have higher R-squared values 

var = ""
for col in significant.keys():
    var += col
    li = list(significant.keys())
    if col == li[-1]:
        break
    var += " + "
total = "lgdp ~ " + var
residual = "resid ~ " + var

result = sm.ols(formula= total, data=df_model).fit()
df_model['yhat'] = result.fittedvalues
df_model['resid'] = result.resid

# Check for Breusch Pagan test for Hetereoskedasticity
result_re = sm.ols(formula=residual, data=df_model).fit()
test = sms.het_breuschpagan(result_re.resid, result_re.model.exog)

# Check for White test for Hetereoskedasticity
result_wh = sm.ols(formula="resid**2 ~ yhat + yhat**2", data=df_model).fit()
white_test = sms.het_white(result_wh.resid,  result_wh.model.exog)

print(result.params)
print(result.summary())
print("")

print("Breusch Pagan test for Hetereoskedasticity")
print(lzip(name, test))
print("White test for Hetereoskedasticity")
print(lzip(name, white_test))

Intercept             15.388134
year                   0.828424
avg_HPI                0.112990
long_term_interest     0.009242
population            -0.252225
fourth                 0.029582
top_5_percent          0.082741
lCPI                   0.417951
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                   lgdp   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 4.289e+04
Date:                Tue, 19 May 2020   Prob (F-statistic):               0.00
Time:                        02:32:25   Log-Likelihood:                 897.41
No. Observations:                 288   AIC:                            -1779.
Df Residuals:                     280   BIC:                            -1750.
Df Model:                           7                                         
Covariance Type:  

In [17]:
# Here even though I separated the dataset, there still seem to be clear signs of heteroskedasticy
# Below I would run the total model again with heteroskedasticity robust statistics

# OLS model with Heteroskedasticity-Robust statistics

In [18]:
# Here since there are clear signs of heteroskedasticity error, i would run the model with all data again 
# with heteroskedasticity-robust statistics

# Test run OLS log(gdp) on all variables
# I also include a Breusch Pagan test and a White for hetereoskedasticity for every variables. 

var = ""
for col in use_cols:
    var += col
    if col == 'lfederal_funds_rates':
        break
    var += " + "
total = "lgdp ~ " + var
residual = "resid ~ " + var

result = sm.ols(formula= total, data=df_model).fit(cov_type='HC3', use_t = True)
df_model['yhat'] = result.fittedvalues
df_model['resid'] = result.resid

print(result.params)
print(result.summary())

Intercept               15.304505
year                     0.907602
month                   -0.012309
CPI_Inflation           -0.002232
avg_HPI                  0.119875
long_term_interest      -0.004985
budget_on_education     -0.030810
population              -0.017703
employed_percent         0.205433
unemployed_percent       0.083192
lowest                   0.025201
second                  -0.002168
third                   -0.065461
fourth                   0.062893
top_5_percent           -0.004957
lCPI                     0.187748
lfederal_funds_rates    -0.036178
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                   lgdp   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.009e+05
Date:                Tue, 19 May 2020   Prob (F-statistic):               0.00
Time:       

# Summary and conclusion

Conclusion: 
- There seem to be clear hetereoskedasticity error for every dataset
- The dataset with variables that have higher R squared seem to be more reliable in terms of homoskedasticity (higher p-value for both BP and White tests)

For these simple OLS regression models, the result for interest rate seem to CONFIRM my initial expectation (negative coefficients) because that means generally decreasing interest rates would increase GDP.
While I expect when during a recession, a drop of interest rate would be a boost to GDP (more business investments)

According to the heteroskedasticity robust statistics, the t score for log(federal funds rates) is -5.32 which is very significant so it seems that a decrease in interest rate can actually increase GDP to a greate extend (1% increase in interest rate would decreass GDP by 3.6%)